<a href="https://colab.research.google.com/github/harshith7823/CS760-Project/blob/utils/utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Refer for importing in colab https://stackoverflow.com/questions/48905127/importing-py-files-in-google-colab

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [37]:
def fill_death_col(clinical_data):
  
  male_obese = (clinical_data['Sex'] == 1) & (clinical_data['BMI'] > 30)
  female_obese = (clinical_data['Sex'] == -1) & (clinical_data['BMI'] > 30)
  male_underweight = (clinical_data['Sex'] == 1) & (clinical_data['BMI'] < 18.5)
  female_underweight = (clinical_data['Sex'] == -1) & (clinical_data['BMI'] < 18.5)
  clinical_data['BMI_reduction_factor'] = 0
  clinical_data.loc[male_obese, 'BMI_reduction_factor'] = 4.2
  clinical_data.loc[female_obese, 'BMI_reduction_factor'] = 3.5
  clinical_data.loc[male_underweight, 'BMI_reduction_factor'] = 4.3
  clinical_data.loc[female_underweight, 'BMI_reduction_factor'] = 4.5

  clinical_data['Tobacco_reduction_factor'] = np.where(clinical_data['Tobacco']==1,10,0)

  #Leave na on rows with no death data, fill the rest with age*365+death days 
  clinical_data['Age_at_death'] = np.where(clinical_data['DEATH [d from CT]'].isna(), np.nan, clinical_data['Age at CT']*365 + clinical_data['DEATH [d from CT]'])
  
  # fill death with avg lifespan - alpha(reduction for tobacco) - beta(reduction for bmi)
  corr_bmi_death = clinical_data['BMI'].corr(clinical_data['Age_at_death'])
  corr_tobacco_death = clinical_data['Tobacco'].corr(clinical_data['Age_at_death'])

  print(corr_bmi_death)
  print(corr_tobacco_death)

  #normalize
  alpha = corr_bmi_death/(corr_bmi_death+corr_tobacco_death)
  beta = corr_tobacco_death/(corr_bmi_death+corr_tobacco_death)

  print(alpha)
  print(beta)
  
  avg_lifespan_male=76.3  
  avg_lifespan_female=81.2

  clinical_data['avg_life'] = np.where(clinical_data['Sex']==1, avg_lifespan_male, avg_lifespan_female)
  clinical_data['avg_life'] = clinical_data['avg_life'] - alpha*(clinical_data['BMI_reduction_factor'])-beta*(clinical_data['Tobacco_reduction_factor'])
  clinical_data['DEATH [d from CT]'].fillna((clinical_data['avg_life']-clinical_data['Age at CT'])*365, inplace=True)
  
  return np.array(clinical_data, dtype=np.float32)

In [3]:
def preprocess_clinical_data(oppScrData, mean=True):
    clinical_data = oppScrData.filter(['BMI','BMI >30', 'Sex', 'Tobacco', 'Met Sx', 'FRAX 10y Fx Prob (Orange-w/ DXA)',
                                'FRAX 10y Hip Fx Prob (Orange-w/ DXA)','FRS 10-year risk (%)', 'DEATH [d from CT]' , 'Age at CT'], axis=1)
    # Replace all _,X,blanks with nan
    clinical_data = clinical_data.replace(r'_', np.nan, regex=True)
    clinical_data = clinical_data.replace(r'X', np.nan, regex=True)
    clinical_data = clinical_data.replace(r'^\s*$', np.nan, regex=True)

    # Fill na in bmi column with mean
    clinical_data['BMI'].fillna(value=clinical_data['BMI'].mean(skipna=True), inplace=True)
    
    # Fill na in bmi>30 column based on bmi col
    clinical_data.loc[clinical_data.BMI>30, 'BMI >30'] = 1
    clinical_data.loc[clinical_data.BMI<=30, 'BMI >30'] = -1
    
    clinical_data['Sex'] = np.where(clinical_data['Sex']=='Male',1,-1)
    clinical_data['Met Sx'] = np.where(clinical_data['Met Sx']=='Y',1,-1) 

    # Treat no data in tobacco as no tobacco usage 
    clinical_data['Tobacco'] = np.where(clinical_data['Tobacco']=='Yes',1,-1) 
    
    #Fill death col
    fill_death_col(clinical_data)

    clinical_data['FRS 10-year risk (%)'] = clinical_data['FRS 10-year risk (%)'].replace("<1", 0.01, regex=True)
    clinical_data['FRS 10-year risk (%)'] = clinical_data['FRS 10-year risk (%)'].replace(">30", 0.30, regex=True)
    clinical_data['FRS 10-year risk (%)'] =  clinical_data['FRS 10-year risk (%)'] * 100
 
    cols_to_be_filled = ['FRAX 10y Fx Prob (Orange-w/ DXA)','FRAX 10y Hip Fx Prob (Orange-w/ DXA)','FRS 10-year risk (%)']
    for c in cols_to_be_filled:
      if mean:  
        clinical_data[c].fillna(value=clinical_data[c].mean(skipna=True), inplace=True)
      else :
        clinical_data[c].fillna(value=clinical_data[c].median(skipna=True), inplace=True)

    return [clinical_data, np.array(clinical_data, dtype=np.float32)]

In [33]:
def normalize_with_ss(ct_data):
  scaler = StandardScaler()
  ct_data=scaler.fit_transform(ct_data)
  return ct_data

In [7]:
def normalize_data(data):
    n = data.shape[1]    
    for i in range(n-1):
      data[:,i] = (data[:,i] - np.min(data[:,i]))/(np.max(data[:,i])- np.min(data[:,i]))
    return data

In [9]:
# oppScrData = pd.read_excel (r'sample_data/OppScrData.xlsx')  

In [28]:
# clinical_data, clinical_data_np = preprocess_clinical_data(oppScrData)

-0.13617739399608175
-0.018215285783995346
0.8820197576080556
0.11798024239194438


In [38]:
# clinical_data_np = fill_death_col(clinical_data)
# ret = normalize_with_ss(clinical_data_np)
# print(ret)

-0.3109889057301494
-0.2173760705006509
0.588587282882852
0.41141271711714805
[[ 1.3627766   1.4275429   1.1249607  ... -0.82441354 -0.3276308
  -0.77756083]
 [ 0.19300698  1.4275429  -0.8889199  ... -0.82441354 -4.7654524
   0.70677465]
 [-0.11482714 -0.7005043  -0.8889199  ... -0.82441354  0.92495024
   1.2824147 ]
 ...
 [ 2.270887    1.4275429  -0.8889199  ... -0.82441354  0.22831532
   0.70677465]
 [-0.29952773 -0.7005043  -0.8889199  ... -0.82441354  0.92495024
   1.2824147 ]
 [-1.3923389  -0.7005043   1.1249607  ... -0.82441354 -0.18079564
  -0.08679067]]
